## Algoritmo para predecir el LTV a 24 meses

### Preparación de los datos

1. **Colección y limpieza de datos:**
    - Asegurarse de que los datos incluyan información relevante como demografía del cliente, historial de compras y métricas de interacción.
    - Limpiar los datos para manejar valores faltantes, outliers y consistencia de formatos.
    - Define y calcula la variable objetivo.

### Ingeniería de características

2. **Creación de características:**
    - Crear características que reflejen el comportamiento del cliente, tales como el gasto total, la cantidad promedio de transacciones, la frecuencia de compra, y el tiempo desde la última compra.
    - Considerar la creación de características en ventanas de tiempo como el gasto total en los últimos 6 o 12 meses.

### Análisis exploratorio de datos (EDA)

3. **Análisis exploratorio:**
    - Analizar los datos para entender patrones y relaciones.
    - Visualizar las distribuciones del gasto y otras métricas relevantes a través de diferentes segmentos de clientes.
    - Agrupar clientes basándose en la fecha de su primera compra y seguir su comportamiento a lo largo del tiempo.


### Modelado predictivo

4. **Modelado predictivo:**
    - Dividir los datos en conjuntos de entrenamiento y prueba.
    - Elegir un modelo de regresión adecuado para predecir un resultado continuo como el LTV.
    - Entrenar el modelo en los datos de entrenamiento utilizando las características desarrolladas.

### Evaluación del modelo

5. **Evaluación del modelo:**
    - Evaluar el modelo en el conjunto de prueba usando métricas adecuadas, como el error cuadrático medio (RMSE) o el error absoluto medio (MAE).
    - Analizar los residuales para asegurar que el modelo funciona bien en todos los segmentos de clientes.

### Interpretación del modelo y aplicación de insights

6. **Interpretación y acción:**
    - Interpretar el modelo para entender qué características son más predictivas del LTV.
    - Usar la salida del modelo para segmentar clientes basándose en su LTV predicho y dirigirles estrategias de marketing específicas.

### Despliegue y monitoreo

7. **Despliegue y monitoreo:**
    - Implementar el modelo en un entorno de producción donde pueda predecir el LTV de nuevos clientes.
    - Monitorear regularmente el desempeño del modelo y actualizarlo según sea necesario para manejar cambios en el comportamiento del cliente o en las condiciones del mercado.

In [1]:
import pandas as pd
import numpy as np

In [2]:
customers = pd.read_csv('tlacuachitos_vip_customers_data.csv')
transactions = pd.read_csv('tlacuachitos_vip_transactions.csv')

In [3]:
customers.head()

,CustomerID,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort
0,1,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31
1,2,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31
2,3,46,57978.753383,3,Bachelor,Finance,Europe,1,2019-05-31
3,4,32,60445.266900,3,High School,Education,South America,1,2021-02-28
4,5,60,57741.870929,5,Bachelor,Entertainment,Asia,0,2018-10-31


In [4]:
transactions.head()

,CustomerID,TransactionDate,TransactionAmount
0,1,2023-10-31,518.444092
1,1,2024-07-31,353.796197
2,1,2024-01-31,38.206591
3,1,2024-06-30,724.929423
4,2,2022-02-28,145.616000


In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           1143 non-null   int64  
 1   Age                  1143 non-null   int64  
 2   Income               1143 non-null   float64
 3   Tenure               1143 non-null   int64  
 4   Education            1143 non-null   object 
 5   Industry             1143 non-null   object 
 6   Geographic Location  1143 non-null   object 
 7   Churn_Risk           1143 non-null   int64  
 8   Cohort               1143 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 80.5+ KB


In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         4346 non-null   int64  
 1   TransactionDate    4346 non-null   object 
 2   TransactionAmount  4346 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 102.0+ KB


In [7]:
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])
customers['Cohort'] = pd.to_datetime(customers['Cohort'])

In [8]:
snapshot_date = transactions['TransactionDate'].max()
snapshot_date

Timestamp('2024-08-31 00:00:00')

In [9]:
customers['customer_tenure'] = (snapshot_date.year - customers['Cohort'].dt.year)*12 + (
    snapshot_date.month - customers['Cohort'].dt.month)
customers.head()

,CustomerID,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure
0,1,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
1,2,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36
2,3,46,57978.753383,3,Bachelor,Finance,Europe,1,2019-05-31,63
3,4,32,60445.266900,3,High School,Education,South America,1,2021-02-28,42
4,5,60,57741.870929,5,Bachelor,Entertainment,Asia,0,2018-10-31,70


In [10]:
df_master = transactions.merge(customers, on='CustomerID')
df_master.head()

,CustomerID,TransactionDate,TransactionAmount,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure
0,1,2023-10-31,518.444092,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
1,1,2024-07-31,353.796197,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
2,1,2024-01-31,38.206591,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
3,1,2024-06-30,724.929423,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
4,2,2022-02-28,145.616000,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36


In [11]:
df_master['customer_tenure_on_transaction'] = (
    df_master['TransactionDate'].dt.year - df_master['Cohort'].dt.year)*12 + (
    df_master['TransactionDate'].dt.month - df_master['Cohort'].dt.month)
df_master.head()

,CustomerID,TransactionDate,TransactionAmount,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure,customer_tenure_on_transaction
0,1,2023-10-31,518.444092,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,2
1,1,2024-07-31,353.796197,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,11
2,1,2024-01-31,38.206591,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,5
3,1,2024-06-30,724.929423,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,10
4,2,2022-02-28,145.616000,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36,6


In [12]:
cltv_24_months = df_master[
    (df_master['customer_tenure'] > 24) &
    (df_master['customer_tenure_on_transaction'] <= 24)
].groupby('CustomerID')['TransactionAmount'].sum().reset_index()
cltv_24_months.head()

,CustomerID,TransactionAmount
0,2,145.616000
1,3,487.285458
2,4,1095.025887
3,5,1685.260152
4,6,728.364381


In [13]:
categorical_features = ['Education', 'Industry', 'Geographic Location']
numerical_features = ['Age', 'Income', 'Tenure']

data_encoded = pd.get_dummies(customers[['CustomerID'] + categorical_features],
                             columns=categorical_features,
                             drop_first=False)

df_data_with_encoded = customers[['CustomerID'] + numerical_features].merge(
    data_encoded, on='CustomerID')

df_to_model = cltv_24_months.merge(df_data_with_encoded, on='CustomerID')
df_to_model.rename(columns={'TransactionAmount':'LTV'}, inplace=True)

df_to_model

,CustomerID,LTV,Age,Income,Tenure,Education_Bachelor,Education_High School,Education_Master,Education_PhD,Industry_Education,Industry_Entertainment,Industry_Finance,Industry_Healthcare,Industry_Technology,Geographic Location_Asia,Geographic Location_Australia,Geographic Location_Europe,Geographic Location_North America,Geographic Location_South America
0,2,145.616000,69,55297.364348,6,True,False,False,False,False,False,False,False,True,False,False,False,False,True
1,3,487.285458,46,57978.753383,3,True,False,False,False,False,False,True,False,False,False,False,True,False,False
2,4,1095.025887,32,60445.266900,3,False,True,False,False,True,False,False,False,False,False,False,False,False,True
3,5,1685.260152,60,57741.870929,5,True,False,False,False,False,True,False,False,False,True,False,False,False,False
4,6,728.364381,25,57132.404622,3,False,False,True,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,1136,189.961932,67,59279.959062,4,False,True,False,False,False,False,True,False,False,True,False,False,False,False
858,1138,2375.255820,28,59996.754751,5,True,False,False,False,False,True,False,False,False,False,False,False,True,False
859,1139,1327.509070,26,45236.402018,8,True,False,False,False,True,False,False,False,False,False,False,True,False,False
860,1141,718.548745,29,57043.387540,5,False,True,False,False,False,False,True,False,False,False,False,False,True,False


In [14]:
import statsmodels.api as sm

In [33]:
df_to_model = df_to_model.astype({col: 'int' for col in df_to_model.select_dtypes('bool').columns})

x=['Tenure', 'Education_Bachelor', 'Education_Master', 'Education_PhD', 
        'Industry_Education', 'Industry_Entertainment', 'Industry_Technology', 'Geographic Location_North America']
y=['LTV']

In [34]:
X=sm.add_constant(df_to_model[x])

model=sm.OLS(df_to_model[y], X)
results= model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    LTV   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     14.78
Date:                Thu, 03 Oct 2024   Prob (F-statistic):           2.21e-20
Time:                        20:18:56   Log-Likelihood:                -6771.0
No. Observations:                 862   AIC:                         1.356e+04
Df Residuals:                     853   BIC:                         1.360e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 